In [1]:
import cv2
import numpy as np
import pandas as pd
import imutils
from PIL import Image
import os

In [2]:
import matplotlib.pyplot as plt

In [7]:
# img = cv2.imread(r"C:\Users\sbali\Downloads\misaligned_images\misaligned_images\0e27f636242845138158f132878a3351.jpg")
# plt.imshow(img)

In [4]:
def aligning_image(img_folder_path , des_path):
    images = os.listdir(img_folder_path)
    for i in images:
        image_name , ext = os.path.splitext(i)
        print(f'Aligning image : {image_name + ext}')
    
        if ext =='.jpg' or '.jpeg':
            
            img = cv2.imread(img_folder_path+'/'+i)
    
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img_gray = cv2.bitwise_not(img_gray) 
            # making background light and text dark
            
            # threshold the image, setting all foreground pixels to
            # 255 and all background pixels to 0
            thresh = cv2.threshold(img_gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

            img_coordinates = np.column_stack(np.where(thresh > 0))
            img_angle = cv2.minAreaRect(img_coordinates)[-1]


            # contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            # contours = sorted(contours, key = cv2.contourArea, reverse = True)
            # largestContour = contours[0]
            # img_angle = cv2.minAreaRect(largestContour)[-1]

            #print("before angle : ", img_angle)
            if img_angle < -45 :
                img_angle = -(90 + img_angle)
            
            elif img_angle > 45:
                img_angle = -(-90 + img_angle)
                
            else:
                img_angle = -img_angle
            
            #print("after angle : ", img_angle)
            
            (h, w) = img.shape[:2]
            center = (w / 2, h / 2)

            mul = 1.0
            Mat = cv2.getRotationMatrix2D(center, img_angle, mul) 

            ####### Performed these operations as wrapaffine was cutting the corners of the image 
            abs_cos = abs(Mat[0,0])
            abs_sin = abs(Mat[0,1])

            b_w = int(h * abs_sin + w * abs_cos)
            b_h = int(h * abs_cos + w * abs_sin)

            Mat[0, 2] += b_w/2 - center[0]
            Mat[1, 2] += b_h/2 - center[1]

            rotated = cv2.warpAffine(img, Mat, (b_w, b_h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
            
            cv2.imwrite(des_path+ '/' + image_name + ext, rotated)
            
            print('---------- Alignment Completed ----------')
            print()

In [6]:
aligning_image(r"C:\Users\sbali\Downloads\misaligned_images\misaligned_images", r"C:\Users\sbali\Downloads\misaligned_images\aligned_images")

In [8]:
###### After alignment 

# img = cv2.imread(r"C:\Users\sbali\Downloads\misaligned_images\aligned_images\0e27f636242845138158f132878a3351.jpg")
# plt.imshow(img)